Blog here: https://wordpress.com/post/brianfarish3.wordpress.com/168

We are here: https://docs.aws.amazon.com/textract/latest/dg/api-async.html

Diagram of this pipeline: https://app.diagrams.net/#Hcodeslp%2Faws_arch_diagrams%2Fmaster%2Ftextract-stepfns.drawio

AWS has this which could be helpful to me in writing my code in the testract JSON to RDS Lambda.

boto3 textract documentation: 
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/textract.html#textract

AWS recommends using an asynchronous function to process multiple page files. I'm going to use step functions. More details on that decision in the blog.

#IAM
The role will be named: step-functions-lambda-role. The tag is "app": "document-parser"
I am creating an IAM role for the Lambda function that will allow Step Functions to access resources. The policy will be called "AllowStepFunctionStateMachineStart". It uses the AWS standard policy AWSLambdaRole, which generates cloudwatch logs. Also I am adding permission to start state machines with this policy: 



In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "states:StartExecution",
            "Resource": "arn:aws:states:us-east-1:415832459288:stateMachine:Doc-Parser-Pipeline"
        }
    ]
}

#STEP FUNCTION
I'm creating a step function called Doc-Parser-Pipeline and attaching the above role to it. I'm not turning on logs now, but may later, as I inevitably mess up. Maybe even tracing to with X-ray. Tagged with app:document-parser.

State machine arn:  arn:aws:states:us-east-1:415832459288:stateMachine:Doc-Parser-Pipeline

# LAMBDA
I'm creating a step function "run-step-function-lambda" tagged with app:document-parser.

Adding environment variable:
STATEMACHINEARN:arn:aws:states:us-east-1:415832459288:stateMachine:Doc-Parser-Pipeline


# S3
Creating document upload bucket. Not public. Tag app:document-parser
Bucket name: 
doc-upload-bucket-doc-parser-app
S3 BUCKET ARN:
arn:aws:s3:::doc-upload-bucket-doc-parser-app

Properties > event notifications > create
Event name: 
trigger-document-parsing-event 

Suffix:
.pdf

Object creation, checking "all object create events". From the Lambda dropdown at the bottom selecting "run-step-functions-lambda"

Creating an S3 bucket for textract's JSON object output objects used during the pipeline. No triggers on it for now.

Name: 
textract-json-output-parser-app

ARN:
arn:aws:s3:::textract-json-output-parser-app

We need a bocket policy on this last bucket: 



In [ ]:
{
    "Id": "Policy1683945477760",
    "Version": "2012-10-17",
    "Statement": [
      {
        "Sid": "Stmt1683945475477",
        "Action": "s3:PutObject",
        "Effect": "Allow",
        "Resource": "arn:aws:s3:::textract-json-output-parser-app/*",
        "Principal": {
          "AWS": [
            "arn:aws:iam::415832459288:role/service-role/doc-to-json-textract-role-7n40j7gg"
          ]
        }
      }
    ]
  }

# LAMBDA

We are going to create three Lambdas ***BUT*** they will not actually have the logic we need in them yet. They are basically placeholders. These are in order that they are invoked in the step function:

Name:
doc-to-json-textract
ARN:
arn:aws:lambda:us-east-1:415832459288:function:doc-to-json-textract

Name:
textract-status-checker
ARN:
arn:aws:lambda:us-east-1:415832459288:function:textract-status-checker

Name:
parse-textract-json-obj-for-rds
ARN:
arn:aws:lambda:us-east-1:415832459288:function:parse-textract-json-obj-for-rds

# Reminder: At this point, all of these (and the initial run-step-functions-lambda also) are placeholders and contain no real logic.

# STEP FUNCTIONS

JSON will be pasted after this block.

Editing - Deleting the default Hello and World states from the "States" key. 
Generate code snippet > Invoke a function
Select fn from dropdown > doc-to-json-textract.

Click copy to copy the invoke JSON and paste into definition code snippet after "States": {. Click format JSON to clean it up.
Rename "Invoke function" to "Start extraction".  Make sure key above is "StartAt": "Start Extraction"

Change Next key to "Next:" "Wait for textract"

Add in Wait State. Change name to "Wait for Textract"

Generate code snippet > Invoke a function
Select fn from dropdown > textract-status-checker.  Change first line of new add from Invoke function to "Check textract status".  Make sure above key in Wait state is "Next": "Check textract status".

Add a code snippet for a Choice state. Will be edited heavily see code below. Don't need a "NOT" block. (The default state later will go back to the Wait state.) Essentially two blocks with "StringEquals": "COMPLETED" or second with "StringEquals": "FAILED". The variable "$.Payload.TextractJobStatus".

Change kv Default to "Default": "Wait for textract"

Generate code snippet > Invoke a function
Select fn from dropdown > parse-textract-json-obj-for-rds.  Change first line of new add from Invoke function to "Parse textract JSON".  Make sure above key in Wait state is "Next": "Parse textract JSON".

In choices COMPLETED block set kv "Next": "Parse textract JSON".

Change last "Next" to kv "End": true

For FAILED Choice block:
Generate snippet > flow control state > Fail state. Copy. Paste after END. Change as you see. Set FAILED block kv "Next": "Textract failed"

Chart here: 
/Users/bfaris96/Desktop/data_engineer/FCC-doc-tables/school_assessments_proj/Screen Shot 2023-05-08 at 3.59.30 PM.png


First pass at "run-step-function-lambda"

In [ ]:
import json
import boto3
import os

def lambda_handler(event, context):
    step_function_client = boto3.client('stepfunctions')
    state_machine_arn = os.environ['STATEMACHINEARN']
    
    # Extract relevant information from the S3 event
    s3_bucket = event['Records'][0]['s3']['bucket']['name']
    s3_object_key = event['Records'][0]['s3']['object']['key']
    
    # Prepare the input for the state machine
    step_state = {
        "s3Bucket": s3_bucket,
        "s3ObjectKey": s3_object_key
    }
    
    # Start the execution of the state machine
    response = step_function_client.start_execution(
        stateMachineArn=state_machine_arn,
        input=json.dumps(step_state)
    )

    return json.dumps(response, default=str)

First pass at "doc-to-json-textract"


In [ ]:
import json
import boto3

def lambda_handler(event, context):
    textract_client = boto3.client('textract')

    # Retrieve input from the state machine
    step_state = event['Input']
    s3_bucket = step_state['s3Bucket']   # this is the initial upload bucket
    s3_object_key = step_state['s3ObjectKey']   # this is the initial upload object

    # Prepare the output bucket
    output_bucket = "textract-json-output-parser-app"

    # Start the Textract job for the uploaded document using StartDocumentTextDetection API for extracting raw text
    start_document_text_detection_response = textract_client.start_document_text_detection(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3_bucket,
                'Name': s3_object_key
            }
        },
        OutputConfig={
            'S3Bucket': output_bucket,
            'S3Prefix': f"{s3_object_key}-text-detection/"
        }
    )

    # Start the Textract job for the uploaded document using StartDocumentAnalysis API for extracting tables
    start_document_analysis_response = textract_client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3_bucket,
                'Name': s3_object_key
            }
        },
        FeatureTypes=['TABLES'],
        OutputConfig={
            'S3Bucket': output_bucket,
            'S3Prefix': f"{s3_object_key}-table-analysis/"
        }
    )

    # Creating keys for the text detection and table analysis output objects
    text_detection_output_prefix = f"{s3_object_key}-text-detection/"
    table_analysis_output_prefix = f"{s3_object_key}-table-analysis/"
    step_state['textDetectionOutputObjectKey'] = f"{text_detection_output_prefix}{s3_object_key}.json"
    step_state['tableAnalysisOutputObjectKey'] = f"{table_analysis_output_prefix}{s3_object_key}.json"

    # Update the step-state with the Textract JobIds
    step_state['textDetectionJobId'] = start_document_text_detection_response['JobId']
    step_state['tableAnalysisJobId'] = start_document_analysis_response['JobId']

    return step_state


First pass at "textract-status-checker"

In [ ]:
import json
import boto3

def lambda_handler(event, context):
    textract_client = boto3.client('textract')

    # Retrieve the Textract JobIds, S3 bucket, and object key from the input event
    step_state = event['Input']['Payload']
    text_detection_job_id = step_state['textDetectionJobId']
    table_analysis_job_id = step_state['tableAnalysisJobId']

    # Check the status of the raw text extraction job
    text_detection_status_response = textract_client.get_document_text_detection(JobId=text_detection_job_id)
    text_detection_status = text_detection_status_response['JobStatus']

    # Check the status of the table extraction job
    table_analysis_status_response = textract_client.get_document_analysis(JobId=table_analysis_job_id)
    table_analysis_status = table_analysis_status_response['JobStatus']

    # Add detection and analysis status to the step state
    step_state['textDetectionStatus'] = text_detection_status
    step_state['tableAnalysisStatus'] = table_analysis_status

    # Add TextractJobStatus key to the step state based on the status of both jobs
    if text_detection_status == 'SUCCEEDED' and table_analysis_status == 'SUCCEEDED':
        step_state['TextractJobStatus'] = 'COMPLETED'
    elif text_detection_status == 'FAILED' or table_analysis_status == 'FAILED':
        step_state['TextractJobStatus'] = 'FAILED'

    # Return the status of both Textract jobs along with the S3 bucket and object key
    return step_state



#IAM
More IAM roles and policies: 

For the role attached to doc-to-json-textract:
Policy name: StartTextractTextDetectAndAnalysisAndGetPutS3

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "textract:StartDocumentTextDetection",
                "textract:StartDocumentAnalysis"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::doc-upload-bucket-doc-parser-app/*",
                "arn:aws:s3:::textract-json-output-parser-app/*"
            ]
        }
    ]
}


For the role attached to textract-status-checker:
Name: GetTextractTextDetectAndAnalysisObjects


In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "textract:GetDocumentTextDetection",
                "textract:GetDocumentAnalysis"
            ],
            "Resource": "*"
        }
    ]
}

Also we need this policy on status-checker too: 
Policy name: S3GetPutListGetLocation

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket",
                "s3:GetBucketLocation"
            ],
            "Resource": [
                "arn:aws:s3:::doc-upload-bucket-doc-parser-app",
                "arn:aws:s3:::doc-upload-bucket-doc-parser-app/*",
                "arn:aws:s3:::textract-json-output-parser-app",
                "arn:aws:s3:::textract-json-output-parser-app/*"
            ]
        }
    ]
}

### I have wondered whether I need to build a database before I createa ll these dfs so that I know where I am going and what I'm pointing to. In some cases I can see that being necessary. Here, though, I think I'm just going to export them to parquet

Before this can be written we have to create the database that it will be put into, otherwise I will have no idea how to parse this. It will also need an IAM role and policy attached to it.

Take  look at this: https://github.com/aws-samples/amazon-textract-response-parser

First pass at parse-textract-json-obj-for-rds:

In [ ]:
# the python will go here it is being developed in the other py files

# PLANNING

I have to take a step back at this point and plan our data structures. Here are the tables we want--the titles and the column names. "Pre" meanse pretest and post means posttest, at beginnind and end of intervention. The top-3 foods were only captured in pretest, but can bea meaningful year over year nonetheless.

## Bixby-2019-menu-analysis-pre
entree, score
(roughly 30-50 rows. These are probably all just going to get aggregated with a sum or a median)

## Bixby-2019-variety-pre
num_entree, num_veg, num_fruit
(one row)

## Bixby-2019-smart-total-pre
smart_total 
(one number out of 60 max)

## Bixby-2019-top-3-pre
entree, veg, fruit, grain 
(three rows max, nullable. Might consider pivoting axes so we have columns 1, 2, 3 and rows entree, veg, fruit, grain)

------

## Bixby-2019-menu-analysis-post
entree, score
(roughly 30-50 rows)

## Bixby-2019-variety-post
num_entree, num_veg, num_fruit
(one row)

## Bixby-2019-smart-total-post
smart_total 
(one number out of 60 max)

(no top3 post)

------

## Within our intermediate lambdas, though I need names for the dataframes that correspond to the tables.

menu_df 
=
Bixby-2019-menu-analysis-pre
entree, score
(roughly 30-50 rows)

variety_df
=
Bixby-2019-prep-methods-pre
num_entree, num_veg, num_fruit
(one row)

smart_score
=
Bixby-2019-smart-total-pre
smart_total 
(one number out of 60 max)

top3_df
=
Bixby-2019-top-3-pre
entree, veg, fruit, grain 
(three rows max, nullable.

In [ ]:
pip install textract-trp
pip install amazon-textract-caller
pip install amazon-textract-textractor

In [ ]:
pip install pandas

In [ ]:
pip install amazon-textract-prettyprinter==0.0.16

In [7]:
pip install python-dotenv


  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
You should consider upgrading via the '/Users/bfaris96/Desktop/data_engineer/FCC-doc-tables/school_assessments_proj/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [90]:
##### USING CALL_TEXTRACT FUNCTION #####
##### JUST GETTING TABLES #####


import time
import boto3
import os
from dotenv import load_dotenv

import json
from trp.t_pipeline import pipeline_merge_tables
import trp.trp2 as t2
from textractcaller.t_call import call_textract, Textract_Features
from textractprettyprinter.t_pretty_print import Textract_Pretty_Print, get_string, get_tables_string, Pretty_Print_Table_Format
from trp.trp2 import TDocument, TDocumentSchema
from trp.t_tables import MergeOptions, HeaderFooterType
import boto3
textract_client = boto3.client('textract', region_name='us-east-1')
load_dotenv()

# s3_bucket = "workspace1-s3-bucket"
# output_bucket = "workspace1-s3-bucket"
# s3_object_key = file

s3_uri_of_documents = os.getenv('2022')
textract_json = call_textract(input_document=s3_uri_of_documents, features=[Textract_Features.FORMS, Textract_Features.TABLES], boto3_textract_client = textract_client)




In [91]:
##### PARSE JSON INTO A DICT OF DFS ####



import pandas as pd
from trp import Document
from textractprettyprinter.t_pretty_print import convert_table_to_list

## This commented out code will show you a nice print out in the notebook of the messed up tables
# from IPython.display import display

# def PrettyPrintTables(textract_json):
#     df = None
#     table_count = 0
#     tdoc = Document(textract_json)
#     for page in tdoc.pages:
#       for table in page.tables:
#         table_count += 1
#         df = pd.DataFrame(convert_table_to_list(trp_table=table))
#         print('Table id:', table.id, 'Row count:', len(df.index))
        
# PrettyPrintTables(textract_json)

# import pandas as pd
# from trp import Document

def textract_to_dataframes(textract_json):
    # Convert the Textract JSON textract_json to a set of nested Python objects that we can iterate over
    doc = Document(textract_json)
    
    df_dict = {}
    table_counter = 1
    
    for page in doc.pages:
        for table in page.tables:
            # Convert table to a list of lists, where each sublist represents a row in the table
            table_rows = [[cell.text for cell in row.cells] for row in table.rows]
            # Create a pandas DataFrame from this table data
            df = pd.DataFrame(table_rows)
            # Add this DataFrame to our dictionary of DataFrames
            df_dict[f'df{table_counter}'] = df
            table_counter += 1
    
    return df_dict

df_dict = textract_to_dataframes(textract_json)


In [66]:
print(df_dict)


{'df1':           0                                  1     2 3
0             School Site Student Enrollment:   600   
1  Percent   Free/Reduced-Price Meals Served:         , 'df2':                                                     0    1
0                                 School Quick Facts    1 
1                               Organizational Chart    2 
2            Cooking for Kids Chef Consult Structure    4 
3                               COVID-19 Adaptations    5 
4   Build Collaborative Relationships: Instruction...   7 
5           Entry Conference & Review of Application    8 
6                 School Site's Meals per Labor Hour   10 
7                                Preparation Methods   11 
8                                      Menu Planning   13 
9                                   Food Procurement   14 
10                                      Meal Service   16 
11                        School Nutrition Promotion   17 
12                        Observations & Assessments

In [1]:
print(df_dict)

NameError: name 'df_dict' is not defined

In [ ]:
"""
Textract.Client.start_document_analysis(**kwargs)

response = client.start_document_analysis(
    DocumentLocation={
        'S3Object': {
            'Bucket': 'string',
            'Name': 'string',
            'Version': 'string'
        }
    },
    FeatureTypes=[
        'TABLES'|'FORMS'|'QUERIES'|'SIGNATURES',
    ],
    ClientRequestToken='string',
    JobTag='string',
    NotificationChannel={
        'SNSTopicArn': 'string',
        'RoleArn': 'string'
    },
    OutputConfig={
        'S3Bucket': 'string',
        'S3Prefix': 'string'
    },
    KMSKeyId='string',
    QueriesConfig={
        'Queries': [
            {
                'Text': 'string',
                'Alias': 'string',
                'Pages': [
                    'string',
                ]
            },
        ]
    }
)





Textract.Client.start_document_text_detection(**kwargs)

response = client.start_document_text_detection(
    DocumentLocation={
        'S3Object': {
            'Bucket': 'string',
            'Name': 'string',
            'Version': 'string'
        }
    },
    ClientRequestToken='string',
    JobTag='string',
    NotificationChannel={
        'SNSTopicArn': 'string',
        'RoleArn': 'string'
    },
    OutputConfig={
        'S3Bucket': 'string',
        'S3Prefix': 'string'
    },
    KMSKeyId='string'
)






Textract.Client.get_document_analysis(**kwargs)

response = client.get_document_analysis(
    JobId='string',
    MaxResults=123,
    NextToken='string'
)






Textract.Client.get_document_text_detection(**kwargs)

response = client.get_document_text_detection(
    JobId='string',
    MaxResults=123,
    NextToken='string'
)


"""

In [120]:
#### PARSING ALL THE DFs HERE ###

import re
import pandas as pd
import json

def process_menu_df(dfs_dict):
    df_new = pd.DataFrame(columns=["WEEK_ENTREE", "score"])

    for df_name, df in dfs_dict.items():
        for _, row in df.iterrows():
            row_str = ' '.join(map(str, row.values))
            matches = re.findall(r'(?:WEEK \d+)?\s*Entrée\s*\d:\s*\d', row_str)
            for match in matches:
                week_entree, score = re.split(r':\s*', match)
                week_entree = week_entree.lower().replace(" ", "")
                df_new = pd.concat([df_new, pd.DataFrame({"WEEK_ENTREE": [week_entree], "score": [score]})], ignore_index=True)

    return df_new

# Call the function
menu_df = process_menu_df(df_dict)

# This will return smart score for 2019, using the full response object.

# def process_smart_score_2019(full_response_object):
#     blocks = full_response_object['Blocks']
#     score = None
#     flag = False
#     for block in blocks:
#         if block['BlockType'] == 'LINE':
#             if flag and block['Text'].isdigit():
#                 score = block['Text']
#                 break
#             if block['Text'] == 'Scorecard Total':
#                 flag = True
#     return score

# smart_score_2019 = process_smart_score_2019(full_resp)



# def process_smart_score_2020(dfs_dict):
#     for df_name, df in dfs_dict.items():
#         # Ensure all column names are strings and convert them to lowercase
#         df.columns = df.columns.astype(str).str.lower()
#         if 'scorecard total' in df.columns:
#             last_cell = df['scorecard total'].str.lower().iloc[-1]
#             smart_score_2020 = last_cell.split(' of ')[0]
#             return int(smart_score_2020)
#     return None

# smart_score_2020 = process_smart_score_2020(df_dict)



def process_prep_df(dfs_dict):
    df_new = pd.DataFrame()
    for df_name, df in dfs_dict.items():
        if "# of" in df.iloc[0, 0]:
            for i in range(len(df.columns)):
                col = df.iloc[:, i]
                if "# of" in col[0]:
                    food = col[0].replace("# of", "").replace("choices daily", "").strip()
                    num = col[1]
                    df_new = pd.concat([df_new, pd.DataFrame({"food": [food], "num_avail": [num]})], ignore_index=True)
    return df_new

variety_df = process_prep_df(df_dict)

#   THis will work for 2020 and later:
def process_food_df(dfs_dict):
    df_new = pd.DataFrame()
    for df_name, df in dfs_dict.items():
        df.columns = df.iloc[0].str.lower().str.strip()
        df = df[1:]
        if "entrees" in df.columns:
            df_new = pd.concat([df_new, df.reset_index(drop=True)], ignore_index=True)
    return df_new

top3_df_2020 = process_food_df(df_dict)

def process_food_df(dfs_dict):
    df_new = pd.DataFrame()
    for df_name, df in dfs_dict.items():
        for i in range(len(df)):
            row_values = df.iloc[i, :].values
            if "Entrees" in row_values or "what are the" in ' '.join(map(str, row_values)).lower():
                cols = df.iloc[i, :].reset_index(drop=True)
                df_new = pd.concat([df_new, df.iloc[i+1:, :].reset_index(drop=True)], ignore_index=True)
                df_new.columns = cols
    return df_new

top3_df_2019 = process_food_df(df_dict)


# print the new dataframes
print("--------------MENU_DF------------\n\n", menu_df, "\n\n")
# print("--------------SMART_SCORE_2019------------\n\n", smart_score_2019, "\n\n")
# print("--------------SMART_SCORE_2020------------\n\n", smart_score_2020, "\n\n")
print("--------------VARIETY_DF------------\n\n", variety_df, "\n\n")
print("--------------TOP3_DF_2019------------\n\n", top3_df_2019, "\n\n")
print("--------------TOP3_DF_2020------------\n\n", top3_df_2020, "\n\n")



--------------MENU_DF------------

      WEEK_ENTREE score
0   week1entrée1     3
1        entrée1     2
2        entrée1     3
3        entrée1     3
4        entrée1     3
5        entrée2     3
6        entrée2     3
7        entrée2     2
8        entrée2     2
9        entrée2     2
10  week2entrée1     3
11       entrée2     2
12       entrée1     3
13       entrée2     2
14       entrée1     3
15       entrée2     3
16       entrée1     4
17       entrée2     2
18       entrée1     2
19       entrée2     2
20  week3entrée1     3
21       entrée2     3
22       entrée1     3
23       entrée2     2
24       entrée1     3
25       entrée2     2
26       entrée1     3
27       entrée2     2
28       entrée1     3
29       entrée2     2 


--------------VARIETY_DF------------

         food num_avail
0     entrée        2 
1  vegetable        2 
2      fruit        1  


--------------TOP3_DF_2019------------

 Empty DataFrame
Columns: []
Index: [] 


--------------TOP3_DF_2020------

2023-05-16 test outputDFs are pasted in this file: /Users/bfaris96/Desktop/data_engineer/FCC-doc-tables/school_assessments_proj/OUTPUT_DFs.md

### 2023-05-13:  Right now the problem we have with the above function arises from the fact that the documents are laid out differently between years in some very small ways that are tricky for textract to parse and so it reverses the order of the entrees some times. This is not great, but is not a problem if we aggregate them. Which we probably will. And may have to because I can't figure out how to do it better. 

# Rewrite parse-textract-json-obj-for-rds to use async TExtract API call.   Here is documentation:    https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/textract/client/get_document_text_detection.html

# in order to have the packages we need for the pipeline we have to create a layer in our lambdas

In [ ]:
mkdir -p python/lib/python3.8/site-packages
cd python/lib/python3.8/site-packages
pip install textract-trp -t .
pip install amazon-textract-caller -t .
pip install pandas -t .
cd ../../../..
zip -r my_python_lib.zip .

In [1]:
#publish a new layer: 
aws lambda publish-layer-version --layer-name textractcaller-trp-package --zip-file fileb://python-layer-file.zip --compatible-runtimes python3.10

aws lambda publish-layer-version --layer-name pandas --zip-file fileb://pandas-layer-file.zip --compatible-runtimes python3.10


#update a function to use the new layer version:
aws lambda update-function-configuration --function-name my-function --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:6

# for each function
aws lambda update-function-configuration --function-name doc-to-json-textract --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 &
aws lambda update-function-configuration --function-name table-validator --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 &
aws lambda update-function-configuration --function-name fail-logger --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 &
aws lambda update-function-configuration --function-name insert-to-prod --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 &
aws lambda update-function-configuration --function-name parse-textract-json-obj-for-rds --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 &


aws lambda update-function-configuration --function-name doc-to-json-textract --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 arn:aws:lambda:us-east-1:336392948345:layer:AWSSDKPandas-Python310:2 > /dev/null
aws lambda update-function-configuration --function-name table-validator --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 arn:aws:lambda:us-east-1:336392948345:layer:AWSSDKPandas-Python310:2 > /dev/null
aws lambda update-function-configuration --function-name fail-logger --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 arn:aws:lambda:us-east-1:336392948345:layer:AWSSDKPandas-Python310:2 > /dev/null
aws lambda update-function-configuration --function-name insert-to-prod --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 arn:aws:lambda:us-east-1:336392948345:layer:AWSSDKPandas-Python310:2 > /dev/null
aws lambda update-function-configuration --function-name parse-textract-json-obj-for-rds --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1 arn:aws:lambda:us-east-1:336392948345:layer:AWSSDKPandas-Python310:2 > /dev/null




# this flag will silence standard output, but allow errors to be printed
> /dev/null

# this flag will run the command in the background and return to the terminal: 
&

#  layer arn:  arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package
# textract layer arn:     arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:1
# pandas layer arn:     arn:aws:lambda:us-east-1:336392948345:layer:AWSSDKPandas-Python310:2





##########
# Bash script to update multiple Lambda functions with a new layer version

# Publish new layer version
response=$(aws lambda publish-layer-version --layer-name my-layer --zip-file fileb://my-layer.zip)

# Extract the LayerVersionArn
LayerVersionArn=$(echo $response | jq -r '.LayerVersionArn')

# List of Lambda functions to update
functions=("doc-to-json-textract" "table-validator" "fail-logger" "insert-to-prod" "parse-textract-json-obj-for-rds")

# Update each function with the new layer version
for function in "${functions[@]}"
do
  if aws lambda update-function-configuration --function-name $function --layers arn:aws:lambda:us-east-1:415832459288:layer:textractcaller-trp-package:4
  then
    echo "Successfully updated $function"
  else
    echo "Failed to update $function"
  fi
done

# Replace function1, function2, function3 with your actual Lambda function names.

SyntaxError: invalid syntax (2942977964.py, line 2)

In [ ]:
# copy file to s3
aws s3 cp localfile.txt s3://bucketname/filename.txt

aws s3 cp /Users/bfaris96/Desktop/data_engineer/FCC-doc-tables/school_assessments_proj/McAlester/Pre-Assessment-McAlester-2021-22.pdf s3://doc-upload-bucket-doc-parser-app/Pre-Assessment-McAlester-2021-22.pdf


# copy directory and contents to s3:
aws s3 cp /path/to/your/directory s3://bucketname/directory --recursive

# delete from s3:
aws s3 rm s3://bucketname/filename.txt

aws s3 rm s3://doc-upload-bucket-doc-parser-app/Pre-Assessment-McAlester-2021-22.pdf

#delete directory and contents from s3:
aws s3 rm s3://bucketname/directory --recursive


In [ ]:
# to increase timeout limit to 10 minutes:  
aws lambda update-function-configuration --function-name table-validator --timeout 600

## TROUBLE SHOOTING THE SECOND HALF OF PARSE DFs: 

In [102]:
print(df_dict)

{'df1':           0                                  1     2 3
0             School Site Student Enrollment:   600   
1  Percent   Free/Reduced-Price Meals Served:         , 'df2':                                                     0    1
0                                 School Quick Facts    1 
1                               Organizational Chart    2 
2            Cooking for Kids Chef Consult Structure    4 
3                               COVID-19 Adaptations    5 
4   Build Collaborative Relationships: Instruction...   7 
5           Entry Conference & Review of Application    8 
6                 School Site's Meals per Labor Hour   10 
7                                Preparation Methods   11 
8                                      Menu Planning   13 
9                                   Food Procurement   14 
10                                      Meal Service   16 
11                        School Nutrition Promotion   17 
12                        Observations & Assessments

In [106]:
import pandas as pd
import numpy as np

# Write DataFrame to "menu.csv" in validation bucket
for df_name, df in df_dict.items():
    if df[0].str.contains('WEEK 1').any():
        # remove first row
        fixed_df = df.copy().drop(df.index[0])
        # Strip extra spaces from column names and cells
        fixed_df = fixed_df.applymap(str.strip)
        fixed_df.columns = fixed_df.columns.astype(str).str.strip()
        fixed_df = fixed_df.drop(fixed_df.index[0])

        # Rename columns
        new_columns = ['WEEK_NUM', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        fixed_df.columns = new_columns[:len(fixed_df.columns)]

        # Replace empty strings with np.nan
        fixed_df.replace("", np.nan, inplace=True)
        
        # Fill missing values in 'WEEK_NUM' column
        fixed_df['WEEK_NUM'].ffill(inplace=True)

        csv_string = fixed_df.to_csv(index=False)
        # s3_client.put_object(Body=csv_string, Bucket=validation_bucket, Key="menu.csv")
        # step_state["menuCsvKey"] = "menu.csv"
        print(csv_string)
        break


# Write DataFrame to "food.csv" in validation bucket
for df_name, df in df_dict.items():
    if df.iloc[0].str.contains('Entrees').any():
        # Strip extra spaces from column names and cells
        df = df.applymap(str.strip)
        df.columns = df.columns.astype(str).str.strip()

        # Set column names to first row's values
        df.columns = df.iloc[0]
        df = df[1:]

        csv_string = df.to_csv(index=False)
        # create the food.csv key with s3ObjectKey
        food_key = f"{s3_object_key}_food.csv"
        # s3_client.put_object(Body=csv_string, Bucket=validation_bucket, Key=food_key)
        # step_state["foodCsvKey"] = food_key
        print(csv_string)
        break


# Write DataFrame to "choices.csv" in validation bucket
for df_name, df in df_dict.items():
    if df[0].str.contains("# of entrée choices daily").any():
        # Strip extra spaces from column names and cells
        df = df.applymap(str.strip)
        df.columns = df.columns.astype(str).str.strip()

        # Set column names to first row's values
        df.columns = df.iloc[0]
        df = df[1:]

        csv_string = df.to_csv(index=False)
        # s3_client.put_object(Body=csv_string, Bucket=validation_bucket, Key="choices.csv")
        # step_state["choicesCsvKey"] = "choices.csv"
        print(csv_string)
        break


WEEK_NUM,Monday,Tuesday,Wednesday,Thursday,Friday
WEEK 1,Entrée 1: 3,Entrée 1: 2,Entrée 1: 3,Entrée 1: 3,Entrée 1: 3
WEEK 1,Entrée 2: 3,Entrée 2: 3,Entrée 2: 2,Entrée 2: 2,Entrée 2: 2
WEEK 2,Entrée 1: 3 Entrée 2: 2,Entrée 1: 3 Entrée 2: 2,Entrée 1: 3 Entrée 2: 3,Entrée 1: 4 Entrée 2: 2,Entrée 1: 2 Entrée 2: 2
WEEK 3,Entrée 1: 3 Entrée 2: 3,Entrée 1: 3 Entrée 2: 2,Entrée 1: 3 Entrée 2: 2,Entrée 1: 3 Entrée 2: 2,Entrée 1: 3 Entrée 2: 2
WEEK 4,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:
WEEK 5,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:
WEEK 6,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:,Entrée 1: Entrée 2:

,,,
,,,
,,,

# of entrée choices daily,# of vegetable choices daily,# of fruit choices daily
2,2,1



In [88]:
%reset

In [ ]:
### Lambda test event for validator:   

{
  "Input": {
    "Payload": {
      "s3Bucket": "doc-upload-bucket-doc-parser-app",
      "validationBucket": "validation-bucket---doc-parser",
      "s3TextractJsonKey": "Pre-Assessment-McAlester-2021-22.pdf_textract.json",
      "s3ObjectKey": "Pre-Assessment-McAlester-2021-22.pdf",
      "menuCsvKey": "Pre-Assessment-McAlester-2021-22.pdf_menu.csv",
      "foodCsvKey": "Pre-Assessment-McAlester-2021-22.pdf_food.csv",
      "choicesCsvKey": "Pre-Assessment-McAlester-2021-22.pdf_choices.csv"
    }
  }
}